In [ ]:
%run National_Park_Codes.ipynb

In [ ]:
import urllib.parse
import urllib.request
import json
import pandas as pd

baseurl = "https://developer.nps.gov/api/v1/activities/parks?"
activities_per_park = []

for code, name in nationalParkDict.items():
    print(f"Fetching activities for {name} ({code})...")
    try:
        # Construct the query for this park
        para_dict = {
            "q": name,
            "api_key": apikey,
            "limit": 50,  # Adjust as needed
        }
        url2check = baseurl + urllib.parse.urlencode(para_dict)
        result = urllib.request.urlopen(url2check).read()
        result_dict = json.loads(result)
        
        # Parse the activities for this park
        for activity in result_dict.get("data", []):
            activity_name = activity.get("name", "Unknown")
            activity_parks = activity.get("parks", [])
            
            # Find if this park is listed under the activity
            matched = False
            for park in activity_parks:
                if park.get("parkCode") == code.lower() or name.lower() in park.get("fullName", "").lower():
                    matched = True
                    activities_per_park.append({
                        "Park": name,
                        "ParkCode": code,
                        "Activity": activity_name
                    })
                    break  # Stop iterating through parks once a match is found

            if not matched:
                print(f"No activities found for {name} in activity {activity_name}.")
    
    except Exception as e:
        print(f"Error fetching activities for {name} ({code}): {e}")
        continue

# Convert to DataFrame for analysis
activities_df = pd.DataFrame(activities_per_park)
activities_df.to_csv('activities_per_park.csv', index=False)
# Display or save the DataFrame
activities_df.head()

